## keyword集約

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import japanize_matplotlib

import MeCab

In [2]:
df = pd.read_csv('../data/1.train_data.csv')

In [3]:
def replace_rule(text):
    words = text.split(" ")
    for i in range(len(words)):
        words[i] = re.sub(r"(する|した|の|犯|者|官|事態|証言)$", "", words[i])
        words[i] = re.sub(r"\S+テロ", "テロ", words[i])
        words[i] = re.sub(r"\S+雷", "雷", words[i])
        words[i] = re.sub(r"\S+衝突", "衝突", words[i])
        words[i] = re.sub(r"\S*殺人\S*", "殺人", words[i])
    words = sorted(list(set(words)))
    return " ".join(words)
    
# 助詞やいらない部分を削除
df['nkeyword'] = df.keyword.apply(replace_rule)
nkeywords = df.nkeyword.to_list()
print(len(nkeywords))
print(len(set(nkeywords)))

20285
272


In [7]:
one_keywords = df.nkeyword[df.nkeyword.apply(lambda x: len(x.split(" "))) == 1].value_counts().to_dict()

In [23]:
print(len(one_keywords))
list(one_keywords.items())[-10:]

78


[('怪我人', 206),
 ('消防', 203),
 ('サイレン', 202),
 ('感染', 200),
 ('震源地', 199),
 ('消防車', 199),
 ('窃盗', 199),
 ('盗撮', 135),
 ('暴力', 134),
 ('地滑り', 114)]

In [16]:
def decide_keyword(keywords: str):
    keyword_list = keywords.split(" ")
    decided = keyword_list[0]
    max_val = one_keywords[decided]
    for k in keyword_list[1:]:
        if max_val < one_keywords[k]:
            decided = k
            max_val = one_keywords[k]
    return decided

df['topic_keyword'] = df.nkeyword.apply(decide_keyword)
df.topic_keyword.head(10)

0      覚醒剤
1        雷
2       血液
3    ハリケーン
4       殺人
5     土砂崩れ
6       警察
7     血まみれ
8     自然災害
9       死傷
Name: topic_keyword, dtype: object

In [21]:
topic_text_pairs = {}
for k in one_keywords.keys():
    topic_text_pairs[k] = df[df.topic_keyword == k].text.to_list()

In [25]:
topic_text_pairs["覚醒剤"][:3]

['【マイリスト】カカカカ覚醒剤 https://t.co/15ZY9Oa2QS #sm36302050',
 '@noooooooorth @TGN54 覚醒剤初犯は無条件執行猶予ですよ。 初犯でも営利目的所持は、厳しいですよ。',
 '沢尻被告はベテランジャンキー？覚醒剤避けた可能性。・・そうなんだ。']

In [36]:
import numpy as np
np.concatenate([df.nkeyword.values,df.nkeyword.values])

array(['覚醒剤', '雷', '血液', ..., '爆発', '緊急', '炎上'], dtype=object)